## Traditional method

In [1]:
from IPython.display import IFrame

powerBiEmbed = 'https://app.powerbi.com/reportEmbed?reportId=b53964b5-4014-4d4b-aa21-e2d7c853b575&autoAuth=true&ctid=47446edc-affd-4e7c-bc93-b2907ae98637&config=eyJjbHVzdGVyVXJsIjoiaHR0cHM6Ly93YWJpLWVhc3QtYXNpYS1hLXByaW1hcnktcmVkaXJlY3QuYW5hbHlzaXMud2luZG93cy5uZXQvIn0%3D'

IFrame(powerBiEmbed, width=800, height=600)

## Latest method

In [2]:
from powerbiclient import Report, models
from io import StringIO
import pandas as pd

In [3]:
# Import the DeviceCodeLoginAuthentication class to authenticate against Power BI
from powerbiclient.authentication import DeviceCodeLoginAuthentication

# Initiate device authentication
device_auth = DeviceCodeLoginAuthentication()
access_token = device_auth.get_access_token()

Performing interactive authentication. Please follow the instructions on the terminal.
 To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code HFKRW25T6 to authenticate.
You have logged in.
Interactive authentication successfully completed.


In [5]:
group_id="da31e657-9dce-4853-a0a9-de55ecd2a11a"
report_id="b53964b5-4014-4d4b-aa21-e2d7c853b575"

In [6]:
report = Report(group_id=group_id, report_id=report_id, access_token = access_token)

In [7]:
def loaded_callback(event_details):
    print('The report successfully loaded!!')
    
def error_callback(event_details):
    print('error', event_details)
    
report.on('loaded', loaded_callback)
report.on('error', error_callback)

In [8]:
report

Report()

The report successfully loaded!!


# Interact with the report data with Pandas, Numpy etc.

In [9]:
active_page = report.get_pages()[0]['name']

In [11]:
visuals = report.visuals_on_page(active_page)

In [12]:
visuals

[{'name': '74c638411f1d149face0',
  'title': 'Count of BIAS_DESC by BIAS_DESC',
  'type': 'columnChart',
  'layout': {'x': 0,
   'y': 123.82284382284382,
   'z': 0,
   'width': 417.7156177156177,
   'height': 271.5151515151515,
   'displayState': {'mode': 0}}},
 {'name': '0495aba3d6ca305e7e48',
  'title': 'Count of OFFENSE_NAME and Current Year by OFFENSE_NAME',
  'type': 'donutChart',
  'layout': {'x': 736.969696969697,
   'y': 453.5198135198135,
   'z': 1000,
   'width': 543.030303030303,
   'height': 213.33333333333334,
   'displayState': {'mode': 0}}},
 {'name': '094d3f42aeba198cd36a',
  'type': 'textbox',
  'layout': {'x': 0,
   'y': 454,
   'z': 2000,
   'width': 373,
   'height': 213,
   'displayState': {'mode': 0}}},
 {'name': 'f067fa5f011935eab287',
  'title': 'INCIDENT_ID by Year',
  'type': 'areaChart',
  'layout': {'x': 474.4055944055944,
   'y': 210.34965034965035,
   'z': 3000,
   'width': 805.5944055944057,
   'height': 146.2004662004662,
   'displayState': {'mode': 0}}}

In [13]:
visual = next(filter(lambda visuals: visuals['type'] == 'areaChart', visuals))

In [14]:
visual_name = visual['name']

In [15]:
visual_name

'f067fa5f011935eab287'

In [16]:
summ_data = report.export_visual_data(active_page, visual_name, rows=51)
data = StringIO(summ_data)

In [17]:
df = pd.read_csv(data, sep=",")

In [18]:
df

,Year,INCIDENT_ID,Earliest Date
0,2014,5599,2014-01-01 00:00:00
1,2015,5879,2015-01-01 00:00:00
2,2016,6273,2016-01-01 00:00:00
3,2017,7325,2017-01-01 00:00:00
4,2018,7189,2018-01-01 00:00:00
5,2019,8028,2019-01-01 00:00:00


In [16]:
%matplotlib inline

In [19]:
def filter_report(BIAS_DESC):
    year_filter = {
        '$schema': "http://powerbi.com/product/schema#advanced",
        'target': {
            'table': "hate_crime-csv-2-csv",
            'column': "BIAS_DESC"
        },
        'operator': "In",
        'values': [BIAS_DESC]
    }
    
    report.remove_filters()
    report.update_filters([year_filter])

In [20]:
from ipywidgets import interact

In [21]:
interact(filter_report, BIAS_DESC = ['Anti-Jewish', 'Anti-White'])

interactive(children=(Dropdown(description='BIAS_DESC', options=('Anti-Jewish', 'Anti-White'), value='Anti-Jew…

<function __main__.filter_report(BIAS_DESC)>

In [23]:
report

Report()

The report successfully loaded!!
